In [15]:
from transformers import AutoTokenizer
import pprint
from collections import Counter

In [2]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokens = tokenizer("Hello world")["input_ids"]
print(tokens)

[15496, 995]


In [9]:
pprint.pprint(vars(tokenizer))

{'_additional_special_tokens': [],
 '_bos_token': '<|endoftext|>',
 '_cls_token': None,
 '_decode_use_source_tokenizer': False,
 '_eos_token': '<|endoftext|>',
 '_in_target_context_manager': False,
 '_mask_token': None,
 '_pad_token': None,
 '_pad_token_type_id': 0,
 '_processor_class': None,
 '_sep_token': None,
 '_tokenizer': <tokenizers.Tokenizer object at 0x563afb171b30>,
 '_unk_token': '<|endoftext|>',
 'add_bos_token': False,
 'add_prefix_space': False,
 'clean_up_tokenization_spaces': True,
 'deprecation_warnings': {},
 'init_inputs': (),
 'init_kwargs': {'add_prefix_space': False,
                 'bos_token': '<|endoftext|>',
                 'eos_token': '<|endoftext|>',
                 'model_max_length': 1024,
                 'name_or_path': 'gpt2',
                 'special_tokens_map_file': None,
                 'unk_token': '<|endoftext|>'},
 'model_input_names': ['input_ids', 'attention_mask'],
 'model_max_length': 1024,
 'name_or_path': 'gpt2',
 'padding_side': 'rig

In [14]:
print(tokenizer.vocab_size)
print(type(tokenizer.vocab))
pprint.pp(tokenizer.vocab)

50257
<class 'dict'>
{'Ġ560': 38089,
 'Ġpalm': 18057,
 'Ġimpecc': 45707,
 'isSpecial': 39714,
 'ĠTus': 46233,
 'ĠCand': 15518,
 'Ġprison': 3770,
 'emouth': 46880,
 'Ġapart': 5475,
 'amental': 6860,
 'ĠTransport': 19940,
 'Ġ92': 10190,
 'ĠLad': 12862,
 'ĠArlington': 30379,
 'Ġideal': 7306,
 '759': 38314,
 'ĠSmithsonian': 40131,
 'AGE': 11879,
 'Ġnight': 1755,
 'Ġguessing': 25260,
 'Emb': 31567,
 'Ġtion': 34222,
 'expression': 38011,
 'ĠTro': 8498,
 'Ġflip': 14283,
 'iants': 17883,
 'olen': 8622,
 'Ġble': 7245,
 'Ġhypocr': 25258,
 '317': 34125,
 'ĠEl': 2574,
 'Ġtractor': 38278,
 'ively': 2280,
 'Ġhypothes': 22079,
 'Ġforc': 24686,
 'ãĥĩ': 21959,
 'Ġprolet': 22758,
 'Ġdim': 5391,
 'storms': 38563,
 'hens': 5135,
 'ĠHebdo': 39936,
 'ĠAgility': 43406,
 'Ġvets': 45024,
 'ive': 425,
 'Ġramp': 10454,
 'Ġevid': 25568,
 'ĠKer': 17337,
 'itbart': 17868,
 'thereal': 37827,
 'Ġwi': 45967,
 'ĠAda': 47395,
 'ĠUNIVERS': 49677,
 'mology': 29126,
 'Prom': 24129,
 'Ġgoverning': 15030,
 'Ġsystem': 1080,
 

In [17]:
counter = Counter(tokenizer.vocab)
print(counter.most_common(10))

[('<|endoftext|>', 50256), ('Ġgazed', 50255), ('Ġinformants', 50254), ('ĠCollider', 50253), ('Ġregress', 50252), ('ominated', 50251), ('Ġamplification', 50250), ('Compar', 50249), ('âĢ¦."', 50248), ('Ġ(/', 50247)]
